<center><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac_logo.svg"></center>
<center><img src="https://img.shields.io/github/license/tquangsdh20/rbac-nist?style=plastic"> <img src="https://img.shields.io/badge/author-Tran%20Quang%20%7C%20Nguyen%20Dinh%20Hoang%20Quy%20%7C%20Than%20Hai%20Nhat%20Minh-blue?style=plastic"> <a href="https://hcmuteduvn-my.sharepoint.com/:p:/g/personal/tquang_sdh20_hcmut_edu_vn/EZmcIMj8739LrdtF9CRbHwoBQ3NGoQUoaDMCjeEF6iZ_iA?e=dzX99T"><img src="https://img.shields.io/badge/PowerPoint-RBAC-red?style=plastic&logo=microsoftpowerpoint&logoColor=orange"> <img src="https://img.shields.io/badge/Python-3.7%7C3.8%7C3.9-blue?style=plastic&logo=python"></a></center>

## Description

- **About**: Demo for the RBAC following NIST Model - Database Security Course Work 
- **School**: HCM City University of Technology 
- **Lucturer** : [Dang Tran Khanh](http://www.cse.hcmut.edu.vn/~khanh/)

## Introduction to RBAC

#### RBAC stand for *Role Based Access Control* model which satisfied the following conditions:
- Access is determined by roles
- A user’s roles are assigned by security administrators
- A role’s permissions are assigned by security administrators

#### For instance:

<img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac_example.svg">

## RBAC Models

#### There are 4 levels of RBAC Model as the followings:

- **RBAC0**: *contains the minimum functionality for an RBAC system*
- **RBAC1**: *includes the RBAC0 functionality and adds role hierarchies, which enable one role to inherit permissions from another role*
- **RBAC2**: *includes RBAC0 and adds constraints, which restrict the ways in which the components of a RBAC system may be configured*
- **RBAC3**: *contains the functionality of RBAC0, RBAC1, and RBAC2*

## The NIST Model for RBAC

### What is the NIST?

<a href="https://vi.wikipedia.org/wiki/Vi%E1%BB%87n_Ti%C3%AAu_chu%E1%BA%A9n_v%C3%A0_K%E1%BB%B9_thu%E1%BA%ADt_qu%E1%BB%91c_gia_(Hoa_K%E1%BB%B3)"><p align="center"><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/nist.svg"></p></a>

### The RBAC following NIST Model

<a href="https://vi.wikipedia.org/wiki/Vi%E1%BB%87n_Ti%C3%AAu_chu%E1%BA%A9n_v%C3%A0_K%E1%BB%B9_thu%E1%BA%ADt_qu%E1%BB%91c_gia_(Hoa_K%E1%BB%B3)"><p align="center"><img src="https://raw.githubusercontent.com/tquangsdh20/RBAC-NIST/main/.github/rbac-nist.svg"></p></a>

**Please refer this file [![PowerPoint](https://img.shields.io/badge/PowerPoint-RBAC-red?style=plastic&logo=microsoftpowerpoint&logoColor=orange)](https://hcmuteduvn-my.sharepoint.com/:p:/g/personal/tquang_sdh20_hcmut_edu_vn/EZmcIMj8739LrdtF9CRbHwoBQ3NGoQUoaDMCjeEF6iZ_iA?e=dzX99T) for more details about RBAC**

<center><a href="https://github.com/tquangsdh20/rbac-nist"><img src="https://img.shields.io/badge/Github-RBAC_NIST-orange?style=social&logo=github"></a></center>

## Implementation

### Generate the policy file for the RBAC Model

In [8]:
import sqlite3
import json
from const import * 

def validation(cursor,query,role,exception):
    cursor.execute(query)
    validList = cursor.fetchall()
    if (role,) not in validList: raise exception

class DB:
    def __init__(self,filename):
        self.conn = sqlite3.Connection(filename)
        self.cur = self.conn.cursor()
        
    def init_database(self):
        self.cur.executescript(INIT_DATABASE)
        self.conn.commit()
    
    def insert_user(self,record):
        self.cur.execute(INSERT_USER)
        
    def reset_roles_for_user(self,user):
        __reset_role__ = "UPDATE users SET role_in_session = '{}' WHERE username=? ;"
        self.cur.execute(__reset_role__,(user,))
        
    def UA(self,user,session,role):
        __GET_USER = "SELECT role_in_session FROM users WHERE username = ? ;"
        __UPDATE_ROLE = "UPDATE users SET role_in_session = ? WHERE username = ?;"
        
        #Validate Users in RBAC Model
        __VALIDATE_USER__ = "SELECT username FROM users;"
        validation(self.cur,
                   __VALIDATE_USER__,
                   user,
                   InValidValue(f"Username '{user}' does not exist in database")
                  )
        
        #Validation Roles in RBAC Model
        __VALIDATE_ROLE__ = "SELECT name FROM roles;"
        validation(self.cur,
                   __VALIDATE_ROLE__,
                   role,
                   InValidValue(f"Role '{role}' does not exist in database")
                  )
        
        #Validation Sessions in RBAC Model
        __VALIDATE_SESSION__ = "SELECT name FROM sessions;"
        validation(self.cur,
                   __VALIDATE_SESSION__,
                   session,
                   InValidValue(f"Session '{session}' does not exist in database")
                  )
        # Role Assignment for User 
        self.cur.execute(__GET_USER,(user,))
        prev_session = json.loads(self.cur.fetchone()[0])
        prev_session[session] = role
        new_session = json.dumps(prev_session)
        self.cur.execute(__UPDATE_ROLE,(new_session,user))
        
    def insert_role(self,record):
        self.cur.execute(INSERT_ROLE)
        
    def get_users(self):
        self.cur.execute(GET_USERS)
        return self.cur.fetchall()
    
    def sign_in(self,username,password):
        self.cur.execute(GET_USER,(username,password))
        return self.cur.fetchone()
    
    def policy_update(self):
        # Get all permisions
        self.cur.execute(GET_PRMS)
        prms = self.cur.fetchall()
        
        # Get all users' roles
        self.cur.execute(GET_USERS)
        users = self.cur.fetchall()
        
        # Write content with formats
        PRM = 'p, {role}, {session}, {obj}, {action}\n'
        GRT = 'g, {user}, {role}, {session}\n'
        
        # Write down all Permisions and all Users' Assignment
        with open('./model/rbac_policy.csv','w') as fp:
            for prm in prms:
                fp.write(PRM.format(role=prm[0],session=prm[1],obj=prm[2],action=prm[3]))
            for user in users:
                username,role_str = user
                roles = json.loads(role_str)
                for session,role in roles.items():
                    fp.write(GRT.format(user=username,role=role,session=session))
            fp.close()
            
    def commit(self):
        self.conn.commit()
        
    def close(self):
        self.conn.commit()
        self.cur.close()
        self.conn.close()
        
db = DB('./model/Database_RBAC.db')
db.init_database()
db.policy_update()

In [9]:
quit()

In [4]:
db.UA('one.sdh20','MATH','STUDENT')
res = db.sign_in('one.sdh20','password')
if res is None:
    print('Error: Password or User is incorrect.')
else:
    user,name,role_str = res
    roles = json.loads(role_str)
print(roles)

{'MATH': 'STUDENT'}


In [1]:
db.reset_roles_for_user('one.sdh20')
res = db.sign_in('one.sdh20','one')
if res is None:
    print('Error: Password or User is incorrect.')
else:
    user,name,role_str = res
    roles = json.loads(role_str)
print(roles)

NameError: name 'db' is not defined

In [3]:
db.close()

### Funtions for access

In [30]:
def retricted(obj):
    print(f'You have no permissions for the {obj}')
    return 

def get_name(obj):
    print(f'You can get info for the {obj}')
    return

def get_grade(obj):
    print(f'You can get the grades from {obj}')
    return 

def write_grade(obj):
    print(f'You can write the grades into {obj}')
    return

def edit_grade(obj):
    print(f'You can edit the grades of the {obj}')
    return 

def edit_info(obj):
    print(f'You can edit information of the records inside {obj}')
    return 
    
actions =  {
    "VIEW NAME":   get_name,
    "VIEW GRADE": get_grade,
    "WRITE GRADE": write_grade,
    "EDIT GRADE": edit_grade,
    "EDIT INFO":  edit_info,
    }

objects =  {
    "MATH" : "math.db",
    "IT" : "it.db",
    "CHEMISTRY" :"chemistry.db",
    }